<a href="https://colab.research.google.com/github/duong-trung/Pytorch-tutorials/blob/main/03_pytorch_classification_up.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>